# Getting music lyrics usign Web Scrapping

This short notebook is designed to automatically obtaining song lyrics from the page [letras.com](www.letras.com) using web scrapping, requiring only the main link of an artist on the page.

## Needed libraries

In [1]:
# The only library you'll need to install is Beautiful Soup 4 (use: "pip install beautifulsoup4").
from bs4 import BeautifulSoup as bs

# The last three libraries are from the python standard library.
import requests     # To make web requests
import time         # To measure time
import json         # To handle json files
import re           # To use regular expressions

## Put here your artist main link from letras.com and its name

In [2]:
url_base = 'https://www.letras.com/imagine-dragons/'

## Concatenating necessary link to acces to the artist song list

In [3]:
url = url_base + 'mais_acessadas.html'
page = requests.get(url)
assert str(page) == '<Response [200]>', 'Bad response from URL'

## Scrapping page content

In [4]:
# Parsing web content with BeautifulSoup
soup = bs(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE HTML>
<html dir="ltr" lang="es">
 <head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb#">
  <title>
   Imagine Dragons - LETRAS.COM (167 canciones)
  </title>
  <meta content='Ver las letras de Imagine Dragons y escuchar "Believer",  "Demons",  "Bad Liar",  "Bones",  "Not Today",  "Natural",  "Thunder",  "Radioactive" y más canciones!' name="description"/>
  <meta content="  letras de músicas, Imagine Dragons letras, músicas, letra de música, letra, letras de música, legendas, vídeos  " name="keywords"/>
  <link href="https://www.letras.com/imagine-dragons/" rel="canonical"/>
  <meta charset="utf-8"/>
  <meta content="app-id=com.studiosol.player.letras" name="google-play-app"/>
  <meta content="214701744179" property="fb:pages">
   <meta content="357922461263649" property="fb:app_id"/>
   <meta content="telephone=no" name="format-detection"/>
   <meta content="#FFFFFF" name="theme-color"/>
   <meta content="async" name="include_mode">
    <meta content="app-id=77334789

## Finding every song tag element in page

In [5]:
songs_items = soup.find_all('li', class_='cnt-list-row')
songs_items[0]

<li class="cnt-list-row -song is-visible" data-artist="Imagine Dragons" data-dns="imagine-dragons" data-id="2876080" data-name="Believer" data-sharetext="Believer de Imagine Dragons" data-shareurl="https://www.letras.com/imagine-dragons/believer/" data-url="believer"> <a class="song-name" href="/imagine-dragons/believer/"><span>Believer</span></a> <div class="song-options"></div> </li>

## Getting all the songs urls from the artist page

In [6]:
def get_songs_list(artist_url: str) -> list[str]:
    """
    Returns a list with all the songs urls of a given artist

    Args:
        artist_url (str): Artist url rfom letras.com

    Returns:
        list[str]: A list containing the songs urls as strings
    """
    url = artist_url + 'mais_acessadas.html'    # Required for this web
    page = requests.get(url)

    # Parsing web content with BeautifulSoup
    soup = bs(page.content, 'html.parser')
    # Looking for al the 'li' items with 'cnt-list-row' class in the html
    songs_items = soup.find_all('li', class_='cnt-list-row')

    # For every "li" item, we extract the "data-shareurl" tag inside
    songs_urls = [song.get("data-shareurl") for song in songs_items]

    return songs_urls

In [7]:
get_songs_list("https://www.letras.com/luis-miguel/")[:3]

['https://www.letras.com/luis-miguel/108649/',
 'https://www.letras.com/luis-miguel/26107/',
 'https://www.letras.com/luis-miguel/26102/']

## Defining function to get the lyric from a song page

In [8]:
def get_lyrics_from_url(song_url: str) -> str:
    """
    Extract the lyrics of a song page

    Args:
        song_url (str): Url of a song

    Returns:
        str: The lyrics of the given song's url
    """    

    song = requests.get(song_url)
    song = bs(song.content, 'html.parser')
    lyric_div = song.find('div', class_='cnt-letra')
    parrafos = lyric_div.find_all('p')

    lyric = ""

    for parrafo in parrafos:
        text = str(parrafo)
        text = text.replace("<br>", "\n")
        text = text.replace("<br/>", "\n")
        text = text.replace("</br>", "")
        text = text.replace("<p>", "")
        text = text.replace("</p>", "\n")
        lyric += text + "\n"

    lyric = lyric[:-2] # Remove extra \n
    
    return lyric

In [9]:
print(get_lyrics_from_url("https://www.letras.com/travis-scott/maria-im-drunk/"))

[Travis Scott]
Made it to LA, yeah
Finally in LA, yeah
Lookin' for the weed though
Trying to make my own dough
Callin' for Maria
Lost without Maria
Might dive in the marina, oh, yeah, oh

So trust me, baby, trust me
Trust me, baby, trust me
Trust me, baby, trust me
I don't mind
Trust me, baby, trust me
Trust me, baby, trust me
Trust me, baby, trust me
I don't mind

Trust me, trust me
Trust me, trust me
Trust me, trust me
Yeah, I don't mind

[Young Thug &amp; Travis Scott]
Travis Scott
(You know)
Thugger Thugger, nigga, ay

Ay, call your friends, let's get drunk
Call your friends, let's get drunk
(Call your friends and let's get drunk)
Call your friends, let's get drunk
(Call your friends and let's get drunk, ay)
Call your friends, let's get drunk
Call your friends, let's get drunk
Call your friends, let's get drunk
(Call your friends and let's get drunk)
Call your friends, let's get drunk
(Call your friends and let's get drunk)
Call your friends, let's get drunk

Twelve more hours left

### Another way: getting the lyrics from a div tag
This will allow us to avoid redundant requests when we want more information about a given song url

In [10]:
def get_lyrics_from_div(div) -> str:
    """
    Extract the lyrics of a div

    Args:
        div (bs.element.Tag): div tag containing the lyrics

    Returns:
        str: The lyrics of the given song
    """    
    parrafos = div.find_all('p')

    lyric = ""

    for parrafo in parrafos:
        text = str(parrafo)
        text = text.replace("<br>", "\n")
        text = text.replace("<br/>", "\n")
        text = text.replace("</br>", "")
        text = text.replace("<p>", "")
        text = text.replace("</p>", "\n")
        lyric += text + "\n"

    lyric = lyric[:-2] # Remove extra \n
    
    return lyric

In [11]:
def get_lyrics_from_div_nobreaklines(div) -> str:
    """
    Extract the lyrics of a div with no break lines in text

    Args:
        div (bs.element.Tag): div tag containing the lyrics

    Returns:
        str: The lyrics of the given song
    """    
    parrafos = div.find_all('p')

    lyric = ""

    # We use regex to extract and clean the lyrics
    for parrafo in parrafos:

        # Alternartive to get lyrics without breaklines
        # Separate camelCase words (e.g. camelCase -> camel Case)
        lyric += re.sub(r"\B([A-Z])", r" \1", parrafo.text)
        
        # Remove extra spaces
        lyric = re.sub(' +', ' ', lyric)

    lyric = lyric[:-2] # Remove extra \n
    
    return lyric

In [12]:
print(get_lyrics_from_url('https://www.letras.com/imagine-dragons/demons/'))

When the days are cold
And the cards all fold
And the saints we see
Are all made of gold

When your dreams all fail
And the ones we hail
Are the worst of all
And the blood's run stale

I wanna hide the truth
I wanna shelter you
But with the beast inside
There's nowhere we can hide

No matter what we breed
We still are made of greed
This is my kingdom come
This is my kingdom come

When you feel my heat
Look into my eyes
It's where my demons hide
It's where my demons hide
Don't get too close
It's dark inside
It's where my demons hide
It's where my demons hide

At the curtain's call
Is the last of all
When the lights fade out
All the sinners crawl
So they dug your grave
And the masquerade
Will come calling out
At the mess you made

Don't wanna let you down
But I am hell bound
Though this is all for you
Don't wanna hide the truth

No matter what we breed
We still are made of greed
This is my kingdom come
This is my kingdom come

When you feel my heat
Look into my eyes
It's where my demons 

## Joining previous functions to get all the info of a song

In [13]:
def get_song_info(song_url: str) -> dict:
    """
    Returns a dictionary containing important information of a song, such as
    its name, lyrics and url
    
    Args:
        song_url (str): url of song

    Returns:
        dict: Dictionary with the song info
    """

    song = requests.get(song_url)
    song = bs(song.content, 'html.parser')

    # Getting the song name
    song_name = song.find("div", class_="cnt-head_title").find("h1").text

    # Getting song lyrics
    lyric_div = song.find('div', class_='cnt-letra')
    lyrics = get_lyrics_from_div(lyric_div)
    
    # Returning result dict
    return {
        "name": song_name,
        "lyrics": lyrics,
        "url": song_url
    }

In [14]:
print(get_song_info("https://www.letras.com/imagine-dragons/believer/")["lyrics"])

First things first
I'ma say all the words inside my head
I'm fired up and tired of the way
That things have been, oh-ooh
The way that things have been, oh-ooh
Second things second
Don't you tell me what you think that I could be
I'm the one at the sail, I'm the master of my sea, oh-ooh
The master of my sea, oh-ooh

I was broken from a young age
Taking my sulking to the masses
Writing my poems for the few
That look at me, took to me, shook to me, feeling me
Singing from heartache from the pain
Taking my message from the veins
Speaking my lesson from the brain
Seeing the beauty through the

Pain!
You made me a, you made me a believer, believer
Pain!
You break me down and build me up, believer, believer
Pain!
Oh, let the bullets fly, oh, let them rain
My life, my love, my drive, they came from
Pain!
You made me a, you made me a believer, believer

Third things third
Send a prayer to the ones up above
All the hate that you've heard
Has turned your spirit to a dove, oh-ooh
Your spirit up ab

## Get artist songs

In [15]:
def get_artist_songs(artist_url: str) -> list[dict]:
    """
    Returns name, lyrics and url of every song of an artist url in JSON format
    
    Args:
        artist_url (str): letras.com main page artist url

    Returns:
        list[dict]: List of dictionary with the info (JSON format)
    """
    urls = get_songs_list(artist_url)

    data = [get_song_info(url) for url in urls]
    return data

## Scrapping artist songs concurrently

In [16]:
def scrap_songs_concurrently(artist_url: str, output_name: str) -> int:
    """
    Creates a json file with the songs info of an artist.
    Returns the number of songs scrapped.

    Args:
        artist_url (str): letras.com main page artist url
        output_name (str): name of created json file

    Returns:
        int: Number of songs scrapped.
    """
    
    songs_urls = get_songs_list(artist_url)
    songs_info = [get_song_info(url) for url in songs_urls]

    with open(f"{output_name}.json", "w") as fp:
        json.dump(songs_info, fp, indent=4)

    return len(songs_info)

In [17]:
start_time = time.time()
num_songs = scrap_songs_concurrently("https://www.letras.com/imagine-dragons/", "imagine_dragons_test2_concurrently")
end_time = time.time()
print(f"It took {round(end_time-start_time, 2)} seconds to get {num_songs} songs using Concurrently.")

It took 58.14 seconds to get 167 songs using Concurrently.


## Scrapping artist songs with Threading

In [18]:
import concurrent.futures

def scrap_songs_threading(artist_url: str, output_name: str) -> int:
    """
    Creates a json file with the songs info of an artist using multithreading.
    Returns the number of songs scrapped.

    Args:
        artist_url (str): letras.com main page artist url
        output_name (str): name of created json file

    Returns:
        int: Number of songs scrapped.
    """

    songs_urls = get_songs_list(artist_url)
    # songs_info = []

    # def insert_song_info(url):
    #     songs_info.append(get_song_info(url))

    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(get_song_info, songs_urls)

        songs_info = [result for result in results]

    with open(f"{output_name}.json", "w") as fp:
        json.dump(songs_info, fp, indent=4)

    return len(songs_info)

In [19]:
start_time = time.time()
num_songs = scrap_songs_threading("https://www.letras.com/imagine-dragons/", "imagine_dragons_test2_threading")
end_time = time.time()
print(f"It took {round(end_time-start_time, 2)} seconds to get {num_songs} songs using Multithreading.")

It took 15.46 seconds to get 167 songs using Multithreading.


## Scrapping artist songs with Multiprocessing

In [20]:
import time

In [21]:
#? It is necessary to do it this way to avoid BrokenProcessPool exception
import scrap
import concurrent.futures

if __name__ == "__main__":
    start_time = time.time()
    songs_urls = scrap.get_songs_list("https://www.letras.com/imagine-dragons/")

    with concurrent.futures.ProcessPoolExecutor() as executor:
        results = executor.map(scrap.get_song_info, songs_urls)

        songs_info = [result for result in results]

    with open("imagine_dragons_test2_multiprocessing.json", "w") as fp:
            json.dump(songs_info, fp, indent=4)

    end_time = time.time()
    print(f"It took {round(end_time-start_time, 2)} seconds to get {len(songs_info)} songs using Multiprocessing.")

It took 17.66 seconds to get 167 songs using Multiprocessing.
